In [3]:
import geopandas as gp
import pandas as pd
import numpy as np
from datetime import datetime

from shapely.geometry import box
from shapely.ops import cascaded_union, unary_union

from elasticsearch import Elasticsearch, helpers
ES_DEV = Elasticsearch(['YOUR ES HOST'], http_auth=('ES LOGIN', 'ES PASS'), timeout=30)
%load_ext autotime

/home/nhat/anaconda3/envs/py39/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.0-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


time: 236 µs (started: 2022-03-30 02:58:35 -04:00)


In [16]:
SF52 = { '01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', 
                      '06': 'CA', '08': 'CO', '09': 'CT', '10': 'DE', 
                      '11': 'DC', '12': 'FL', '13': 'GA', '15': 'HI', 
                      '16': 'ID', '17': 'IL', '18': 'IN', '19': 'IA', 
                      '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME', 
                      '24': 'MD', '25': 'MA', '26': 'MI', '27': 'MN', 
                      '28': 'MS', '29': 'MO', '30': 'MT', '31': 'NE', 
                      '32': 'NV', '33': 'NH', '34': 'NJ', '35': 'NM', 
                      '36': 'NY', '37': 'NC', '38': 'ND', '39': 'OH', 
                      '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI', 
                      '45': 'SC', '46': 'SD', '47': 'TN', '48': 'TX', 
                      '49': 'UT', '50': 'VT', '51': 'VA', '53': 'WA', 
                      '54': 'WV', '55': 'WI', '56': 'WY', '72': 'PR'}

SF52R = {v: k for (k,v) in SF52.items()}
CRS_OOKLA = 4326
ookla_zips_dir = 'ookla_zips/'
ookla_tiles_dir = 'ookla_state_tiles/'

def get_tile_url(service_type: str, year: int, q: int) -> str:
    if not 1 <= q <= 4:
        raise ValueError("Quarter must be within [1, 2, 3, 4]")
    month = [1, 4, 7, 10]
    dt = datetime(year, month[q - 1], 1)
    base_url = "https://ookla-open-data.s3-us-west-2.amazonaws.com/shapefiles/performance"
    url = f"{base_url}/type%3D{service_type}/year%3D{dt:%Y}/quarter%3D{q}/{dt:%Y-%m-%d}_performance_fixed_tiles.zip"
    zip_name = f"{dt:%Y-%m-%d}_performance_fixed_tiles.zip"
    return url, zip_name


time: 2.19 ms (started: 2022-03-30 03:12:16 -04:00)


# From ookla_global_tiles, to OOKLA_TILES_BY_STATE

## Ookla state tiles

In [ ]:
# RUN ONCE: slow: download ookla zip files
for QUARTER in ['2021Q1', '2021Q2', '2021Q3', '2021Q4']:
    tile_url, zip_name = get_tile_url("fixed", int(QUARTER[:4]), int(QUARTER[-1:])) 
    print(tile_url, zip_name)
    ! wget {tile_url} -P {ookla_zips_dir}

In [85]:
# TIGER 2019: US boundary, and state boundaries
census_year = 2019
states_url = f"https://www2.census.gov/geo/tiger/TIGER{census_year}/STATE/tl_{census_year}_us_state.zip"
STATE_BOUNDARIES = gp.read_file(states_url)

state_bboxes = {}
for sf in SF52:
    # tiger state shapes has CRS = 4269 >>  Reproject to match the ookla tiles' crs
    xmin, ymin, xmax, ymax = STATE_BOUNDARIES.loc[STATE_BOUNDARIES['STATEFP'] == sf].to_crs(CRS_OOKLA).total_bounds
    # state_bboxes: dictionary mapping sf to polygons (state boundaries)
    state_bboxes[sf] = box(xmin, ymin, xmax, ymax)
    
us_xmin, us_ymin, us_xmax, us_ymax = unary_union(list(state_bboxes.values())).bounds
print(us_xmin, us_ymin, us_xmax, us_ymax)

-179.231086 17.831518837698233 179.85968107125612 71.439786
time: 2.84 s (started: 2022-03-30 00:11:51 -04:00)


In [ ]:
# Choose needed quarter
for QUARTER in [
    '2021Q1', 
    '2021Q2', 
    '2021Q3', 
    '2021Q4',
]:
    tile_url, zip_name = get_tile_url("fixed", int(QUARTER[:4]), int(QUARTER[-1:])) 
    print("READING ZIP FILE ", tile_url, zip_name)
    # Read the downloaded zip: slow read
    ookla_global_tiles = gp.read_file(f'{ookla_zips_dir}{zip_name}')
    # Filter to US ookla tiles
    ookla_us_tiles = ookla_global_tiles.cx[us_xmin:us_xmax, us_ymin:us_ymax]
    # MOST OOKLA speeds were tested within the U.S.
    print(QUARTER, ookla_global_tiles.shape, ookla_us_tiles.shape) 
    
    for sf in SF52:
        # Get state-wise ookla tiles 
        state_xmin, state_ymin, state_xmax, state_ymax = state_bboxes[sf].bounds
        ookla_state_tiles = ookla_us_tiles.cx[state_xmin:state_xmax, state_ymin:state_ymax]
        print(QUARTER, sf, SF52[sf], state_xmin, state_ymin, state_xmax, state_ymax, ookla_state_tiles.shape)

        # Save to individual state tiles: slow write
        file_path = f"{ookla_tiles_dir}{QUARTER}_state_{sf}"
        if len(ookla_state_tiles):
            ookla_state_tiles.to_file(f"{file_path}.geojson", driver="GeoJSON")
        else:    
            print("ALERT: UNLIKELY EVENT: entire states contain no ookla records; creating empty files")
            with open(f"{file_path}.txt", 'a') as f: pass   


## parallel ookla_gen_tiles.py inside cmds.bash: 
* Using command: bash ~/speed/cmds.bash &
* NOTE: Select QUARTER[s] as needed!
* ookla_gen_tiles.py does:
    - cleaning tests in ookla_state_tiles 
    - join state_ookla_tiles with CBG and CB boundaries
    - weight ookla tiles to files: {ookla_tiles_dir}{QUARTER}_{census_level}_{sf}.csv

In [26]:
# COMPLETION CHECK
# Each state now should have 12 files in ookla_state_tiles
# (CBG.csv + CB.csv + state.geojson) x 4 quarter = 12
# 12 * sf52 = 624 files in total in ookla_state_tiles
total = ! ls ookla_state_tiles | wc -l
print(total)

# Find sf with missing ookla files
for sf in SF52:
    count = ! ls ookla_state_tiles/*_{sf}.* | wc -l
    if count[0] != '12':
        print(sf, SF52[sf], count)
        files = ! ls ookla_state_tiles/*_{sf}.*
        print(files)

['624']
time: 395 ms (started: 2022-03-30 03:50:55 -04:00)


In [28]:
# similarly, ESdown.py should generate (ntia + mlab_prediction) * SF52 = 104 files
! ls Elasticsearch/ | wc -l

104
time: 117 ms (started: 2022-03-30 03:51:48 -04:00)
